In [1]:
import pandas as pd

df_account = pd.read_csv('Account.csv')
df_aircraft = pd.read_csv('Aircraft.csv')
df_asset = pd.read_csv('Asset.csv')
df_contact = pd.read_csv('Contact.csv')
df_invoices = pd.read_csv('Invoices.csv')
df_opportunity = pd.read_csv('Opportunity.csv')

print(df_account.head())
print(df_aircraft.head())
print(df_asset.head())
print(df_contact.head())
print(df_invoices.head())
print(df_opportunity.head())

                   Id  Fl3xx_Id__c                             Name  \
0  001OF00000IfKIFYA3       178063  McCooeye Wealth Management Inc.   
1  001OF00000JN4nzYAD       113841              JFK Industries Inc.   
2  001OF00000JQeDQYA1       152871                     Gigadat Inc.   
3  001OF00000IhzM4YAJ       152908                        Don Pratt   
4  001OF00000IGTLUYA5       189375         Conconi FT Holdings Ltd.   

   Primary_Contact__c Aircraft_Type_Owned__c Aircraft_Ownership__c  \
0  003OF00000FkZyIYAV            Praetor 500              25 Hours   
1  003OF000002i3ykYAA            Praetor 500              50 Hours   
2  003OF00000GdiUcYAJ            Praetor 500              75 Hours   
3  003OF00000E57LlYAJ             Legacy 450              50 Hours   
4  003OF00000AtvCnYAJ          Citation CJ2+              25 Hours   

  Lease_Renewal_Date__c Aircraft_Type_Owned_2_c Aircraft_Ownership_2__c  \
0         3/3/2025 0:00                     NaN                     NaN   
1 

In [2]:
import requests
from datetime import datetime
import pandas as pd

# Define the API URL and query parameters
url = 'https://test.fl3xx.com/api/external/flight/flights'
query = {'from': '2025-03-01', 'to': '2025-03-31', 'value': 'ALL', 'timeZone': 'UTC'}
headers = {'X-Auth-Token': '4y9lRGIoQtzDdPLphsO4rtzhTbLPk6Sg'}

# Send the API request
response = requests.get(url, headers=headers, params=query)

# Parse the JSON response
data = response.json()

# The list of relevant flight IDs
relevant_flight_ids = [79952, 79953, 79956, 79957, 79958, 79959, 79960]

# Function to calculate flight time
def calculate_flight_time(etd, eta):
    # Convert ETD and ETA to datetime objects (handling milliseconds with the correct format)
    etd_time = datetime.strptime(etd, '%Y-%m-%dT%H:%M:%S.%fZ')  # Format to include milliseconds
    eta_time = datetime.strptime(eta, '%Y-%m-%dT%H:%M:%S.%fZ')
    
    # Calculate flight duration
    flight_duration = eta_time - etd_time
    return round(flight_duration.total_seconds() / 3600, 1)  # Convert seconds to hours and round to 1 decimal place

# List to store flight data
flight_times = []

# Loop through the flights and calculate flight time for relevant flight IDs
for flight in data:
    flight_id = flight.get('flightId')
    
    # Check if the flight ID is in the list of relevant flight IDs
    if flight_id in relevant_flight_ids:
        etd = flight.get('etd')  
        eta = flight.get('eta')  
        registration_number = flight.get('registrationNumber')  # Get the registration number (tail number)
        airport_from = flight.get('airportFrom')  # Get the origin airport
        airport_to = flight.get('airportTo')  # Get the destination airport
        
        # Ensure all necessary fields are present
        if etd and eta:
            flight_time = calculate_flight_time(etd, eta)
            flight_times.append({
                'Flight ID': flight_id,
                'ETD': etd,
                'ETA': eta,
                'Registration Number': registration_number, 
                'Airport From': airport_from,  # Include the origin airport
                'Airport To': airport_to,  # Include the destination airport
                'Flight Time (hours)': flight_time  # Include the flight time
            })

# Convert to DataFrame
df_flight_times = pd.DataFrame(flight_times)

# Display the DataFrame
df_flight_times


,Flight ID,ETD,ETA,Registration Number,Airport From,Airport To,Flight Time (hours)
0,79960,2025-03-05T20:00:00.000Z,2025-03-05T21:29:00.000Z,C-FSRX,CYED,US-0222,1.5
1,79953,2025-03-07T19:00:00.000Z,2025-03-07T20:00:00.000Z,C-FASP,KSLC,KLAS,1.0
2,79952,2025-03-12T15:00:00.000Z,2025-03-12T19:31:00.000Z,C-GASR,CYYC,CYYZ,4.5
3,79959,2025-03-13T07:00:00.000Z,2025-03-13T07:44:00.000Z,C-GBAS,None,KREG,0.7
4,79956,2025-03-13T16:00:00.000Z,2025-03-13T16:24:00.000Z,C-FSRX,KLAS,CYVR,0.4
5,79958,2025-03-28T23:00:00.000Z,2025-03-29T00:25:00.000Z,C-FNAS,CYXE,CYLW,1.4


In [3]:
# Define the API URL and query parameters for a specific flight ID (79970)
url = 'https://test.fl3xx.com/api/external/flight/flights'
query = {'flightIds': '79970', 'timeZone': 'UTC'}
headers = {'X-Auth-Token': '4y9lRGIoQtzDdPLphsO4rtzhTbLPk6Sg'}

# Send the API request
response = requests.get(url, headers=headers, params=query)

# Parse the JSON response
try:
    data = response.json()
    # Check the response type
    if isinstance(data, list) and data:
        print(f'Flight Data for Flight ID 79970: {data[0]}')
    else:
        print('No data found for Flight ID 79970 or incorrect response format.')
except Exception as e:
    print('Error while parsing JSON:', e)


No data found for Flight ID 79970 or incorrect response format.


In [4]:
# Putting them in a dictionary
dfs = {
    'df_account': df_account,
    'df_aircraft': df_aircraft,
    'df_asset': df_asset,
    'df_contact': df_contact,
    'df_invoices': df_invoices,
    'df_opportunity': df_opportunity,
    'df_flight_times': df_flight_times,
}

# Checking the size (row and column count) of each dataframe
df_sizes = {name: df.shape for name, df in dfs.items()}

# Checking for missing values in each dataframe
df_nulls = {name: df.isnull().sum() for name, df in dfs.items()}

# Checking for duplicate rows in each dataframe
df_duplicates = {name: df.duplicated().sum() for name, df in dfs.items()}

# Display results
df_sizes, df_nulls, df_duplicates

({'df_account': (20, 13),
  'df_aircraft': (40, 2),
  'df_asset': (1335, 5),
  'df_contact': (151, 2),
  'df_invoices': (2006, 9),
  'df_opportunity': (64, 6),
  'df_flight_times': (6, 7)},
 {'df_account': Id                           0
  Fl3xx_Id__c                  0
  Name                         0
  Primary_Contact__c           0
  Aircraft_Type_Owned__c       0
  Aircraft_Ownership__c        0
  Lease_Renewal_Date__c        0
  Aircraft_Type_Owned_2_c     16
  Aircraft_Ownership_2__c     16
  Lease_Renewal_Date_2__c     16
  Aircraft_Type_Owned_3__c    18
  Aircraft_Ownership_3__c     18
  Lease_Renewal_Date_3__c     18
  dtype: int64,
  'df_aircraft': Registration__c      0
  Aircraft_Model__c    0
  dtype: int64,
  'df_asset': Opportunity__c     426
  Name                 0
  AccountId            0
  Ownership__c         0
  Annual_Hours__c      0
  dtype: int64,
  'df_contact': Id      0
  Name    0
  dtype: int64,
  'df_invoices': INVUNIQ          0
  ORDNUMBER        0
  CUST

In [5]:
# Check duplicates since there is a 2 reported
df_asset_duplicates = df_asset[df_asset.duplicated()]
print(df_asset_duplicates)

     Opportunity__c  Name           AccountId  Ownership__c  Annual_Hours__c
876             NaN  GASR  001OF00000IhzM4YAJ         0.090             25.0
1142            NaN  FCXL  001OF00000Ip5kvYAB         0.015             50.0


In [6]:
# Not reporting accurate result, it's natural to have duplicate, need to re-evaluate on the primary key column of each table

columns_to_check = {
    'df_account': 'Id',
    'df_aircraft': 'Registration__c',
    'df_asset': 'Opportunity__c',
    'df_invoices': 'INVUNIQ',
    'df_opportunity': 'Id',
    'df_flight_times': 'Flight ID'
}

# Loop through each dataframe and check for duplicates, counting NaN values
df_duplicate_counts = {}

for df_name, column in columns_to_check.items():
    df_duplicate_counts[df_name] = dfs[df_name][column].duplicated().value_counts()

# Display results
df_duplicate_counts

{'df_account': Id
 False    20
 Name: count, dtype: int64,
 'df_aircraft': Registration__c
 False    40
 Name: count, dtype: int64,
 'df_asset': Opportunity__c
 False    880
 True     455
 Name: count, dtype: int64,
 'df_invoices': INVUNIQ
 False    2006
 Name: count, dtype: int64,
 'df_opportunity': Id
 False    63
 True      1
 Name: count, dtype: int64,
 'df_flight_times': Flight ID
 False    6
 Name: count, dtype: int64}

In [7]:
df_asset_duplicates = df_asset[df_asset.duplicated(subset=['Opportunity__c'], keep=False)]
df_opportunity_duplicates = df_opportunity[df_opportunity.duplicated(subset=['Id'], keep=False)]
df_asset_duplicates, df_opportunity_duplicates

# Same opportunity with different registration number is understandable, as well as the nulls being duplicated.
# In addition, same ID in df_opportunity makes sense since one won and one didn't
# Overall the dataset passes first check

(          Opportunity__c  Name           AccountId  Ownership__c  \
 0                    NaN  FNAS  001OF00000IGTLUYA5       0.01500   
 1                    NaN  GFSD  001OF00000IfKIFYA3       0.02500   
 3                    NaN  GZAS  001OF00000IkA5HYAV       0.01500   
 5                    NaN  GASK  0011G00000vuKQCQA2       0.03120   
 7                    NaN  FIAS  001OF00000JQeDQYA1       0.03125   
 ...                  ...   ...                 ...           ...   
 1264                 NaN  GFSD  001OF00000JQeDQYA1       0.03125   
 1292  006OF00000F7NAXYA3  FASW  001OF00000IVUNgYAP       0.09000   
 1293  006OF00000F7NAXYA3  GASR  001OF00000Ip5kvYAB       0.09000   
 1328  006OF00000IoHcoYAF  FASQ  001OF00000IGTLUYA5       0.09000   
 1329  006OF00000IoHcoYAF  GASK  001OF00000Ioq13YAB       0.09000   
 
       Annual_Hours__c  
 0               125.0  
 1               125.0  
 3                75.0  
 5                25.0  
 7                25.0  
 ...               .

In [8]:
# Summary Visual for Jim Jets

# 1. Account Name - Jim Jets - We can pull up other information from df_account
df_jimjets_account = df_account[df_account['Name'] == 'Jim Jets']
account_name = df_jimjets_account['Name'].iloc[0]
print(account_name)

# 2. Contact Name - we can look up Primary_Contact__c from df_jimjets_account in df_contact - Robert Williams
contact_id = df_jimjets_account['Primary_Contact__c'].values[0]
contact_name = df_contact[df_contact['Id'] == contact_id]['Name'].values[0]
print(contact_name)

# 3. Account Number / Fl3xx ID - 24882 - can be extracted from df_account
account_number = df_jimjets_account['Fl3xx_Id__c'].values[0]
print(account_number)

# 4. Date the summary was created - dynamic date
today_date = datetime.today().date()
print(today_date)

# 5. Anniversary Date - one for CJ2+ and one for CJ3+ 
anniversary_date_cj2 = pd.to_datetime(df_jimjets_account['Lease_Renewal_Date_2__c'].values[0]).strftime('%Y-%m-%d')
anniversary_date_cj3 = pd.to_datetime(df_jimjets_account['Lease_Renewal_Date_3__c'].values[0]).strftime('%Y-%m-%d')
print(anniversary_date_cj2)
print(anniversary_date_cj3)

Jim Jets
Robert Williams
24882
2025-03-23
2025-12-31
2025-05-27


In [9]:
#jj_aggregate_df


In [10]:
# 6. Aircraft - We will use accountID '0011G00000vibZ3QAI'
df_aircraft['Aircraft'] = df_aircraft['Registration__c'] + ' - ' + df_aircraft['Aircraft_Model__c']
df_asset['Registration__c'] = 'C-' + df_asset['Name']
df_combined = df_asset.merge(
    df_aircraft,
    on='Registration__c',
    how='inner')
relevant_df = df_combined[['Aircraft','Ownership__c','Annual_Hours__c','AccountId']]
jj_df = relevant_df[relevant_df['AccountId'] == '0011G00000vibZ3QAI']
jj_aggregate_df = jj_df.groupby('Aircraft', as_index=False)[['Ownership__c','Annual_Hours__c']].sum()
aircraft_cj2 = jj_aggregate_df['Aircraft'].iloc[1]
aircraft_cj3 = jj_aggregate_df['Aircraft'].iloc[10]
print(aircraft_cj2)
print(aircraft_cj3)

# 7. Aircraft Group - If aircraft contains CJ, it's CJ, else Legacy
jj_aggregate_df['AirGroup'] = jj_aggregate_df['Aircraft'].apply(
    lambda x: 'CJ' if 'CJ' in x else 'Legacy'
)
jj_aggregate_df.head()
airgroup_cj2 = jj_aggregate_df['AirGroup'].iloc[1]
airgroup_cj3 = jj_aggregate_df['AirGroup'].iloc[10]
print(airgroup_cj2)
print(airgroup_cj3)


# 8. Ownership Percentage - convert Ownership to percentage and limit to 2 decimal places
jj_aggregate_df['Ownership__c'] = jj_aggregate_df['Ownership__c'] * 100
jj_aggregate_df['Ownership__c'] = jj_aggregate_df['Ownership__c'].apply(lambda x: f'{x:.2f}%')
jj_aggregate_df.head()

owner_cj2 = jj_aggregate_df['Ownership__c'].iloc[1]
owner_cj3 = jj_aggregate_df['Ownership__c'].iloc[10]
print(owner_cj2)
print(owner_cj3)

C-FASP - Citation CJ2+
C-FNAS - Citation CJ3+
CJ
CJ
15.25%
9.00%


In [11]:
# 9. Total Contract Hours Brought Forward from Previous Year - 
# Assuming all of Jim Jets' partial owned CJ2+ aircraft share the same anniversary date
# However, at this point, the ownership hour from df_account does not make sense since it does not align with df_asset
# My assumption is df_account is Aircraft type (CJ2+, CJ3+) ownership hours, and that can be added to individual aircraft ownership hours
# Just want to clarify if this was for customer, I would have made sure everything is clear and triple-checked before handing on customer
# But this is an assessment, my goal is to showcase my skill to the company in bid for an interview.

jj_opportunity_df = df_opportunity[df_opportunity['AccountId'] == '0011G00000vibZ3QAI']
jj_opportunity_df = jj_opportunity_df[jj_opportunity_df['Aircraft_Type__c'].str.contains('CJ')]
#print(jj_opportunity_df)
#print(anniversary_date_cj2)
#print(anniversary_date_cj3)
# CJ2+ contract ran from 12/31/2024 - 12/31/2025
# CJ3+ contract ran from 5/27/2024 - 12/31/2025
# Opportunity shows within the year, Jim Jets added 50 hours of CJ2+ and 25 hours of CJ3+
# However, I am still not seeing anything from the previous year at the moment, which leads me to believe df_asset
# is actually showing rolled over hour from last year or df_account is 
import sqlite3

conn = sqlite3.connect('airsprint_data.db')
cursor = conn.cursor()
df_flight_times.to_sql('flight_times', conn, if_exists='replace', index=False)
df_aircraft.to_sql('aircraft', conn, if_exists='replace', index=False)
conn.commit()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)
conn.close()

conn = sqlite3.connect('airsprint_data.db')
query = """
SELECT
    ft.*, 
    a.* 
FROM 
    flight_times AS ft
INNER JOIN 
    aircraft AS a
ON 
    ft."Registration Number" = a."Registration__c"
"""
df_ft = pd.read_sql(query, conn)
print(df_ft)
conn.close()

df_ft_cj_filtered = df_ft[df_ft['Aircraft_Model__c'].str.contains('CJ', na=False)]
#print(df_ft_cj_filtered)

# Basically, all I need to do is extract information from jj_aggregate_df that contains C-FASP, C-GASR, and C-FNAS 
# I am going to do this manually since I brought over the concactenated column too early
#jj_aggregate_df

annual_hours_cj2 = jj_aggregate_df['Annual_Hours__c'].iloc[1]
annual_hours_cj3 = jj_aggregate_df['Annual_Hours__c'].iloc[10]
print(annual_hours_cj2)
print(annual_hours_cj3)

# C-FASP - Citation CJ2+ | 15.25% ownership | 62.5 Annual Hours | CJ Group
# C-GASR - Citation CJ2+ | 27.38% ownership | 225.0 Annual Hours | CJ Group
# C-FNAS - Citation CJ3+ | 9.00% ownership | 25.0 Annual Hours | CJ Group

# Now I am still a bit lost at where roll over amount from last year
# I will assume that these annual hours from asset for individual registration numbers are rolled over ones
# Summary Visual: An Invoice Summary for JimJets (note – a summary for each Aircraft Group CJ ONLY should be created.
# Thus I should use C-FASP CJ2+ and C-FNAS CJ3+ 

Tables in the database: [('march_2025_filtered',), ('flight_times',), ('aircraft',)]
   Flight ID                       ETD                       ETA  \
0      79960  2025-03-05T20:00:00.000Z  2025-03-05T21:29:00.000Z   
1      79953  2025-03-07T19:00:00.000Z  2025-03-07T20:00:00.000Z   
2      79952  2025-03-12T15:00:00.000Z  2025-03-12T19:31:00.000Z   
3      79959  2025-03-13T07:00:00.000Z  2025-03-13T07:44:00.000Z   
4      79956  2025-03-13T16:00:00.000Z  2025-03-13T16:24:00.000Z   
5      79958  2025-03-28T23:00:00.000Z  2025-03-29T00:25:00.000Z   

  Registration Number Airport From Airport To  Flight Time (hours)  \
0              C-FSRX         CYED    US-0222                  1.5   
1              C-FASP         KSLC       KLAS                  1.0   
2              C-GASR         CYYC       CYYZ                  4.5   
3              C-GBAS         None       KREG                  0.7   
4              C-FSRX         KLAS       CYVR                  0.4   
5              C-F

In [12]:
# 10. Total Hours purchased for 2025
# Jim Jets with won contracts for CJ2+ (25 hours) and CJ3+ (25hours) for 2025
jj_opportunity_df = df_opportunity[df_opportunity['AccountId'] == '0011G00000vibZ3QAI']
jj_opportunity_df = jj_opportunity_df[jj_opportunity_df['Aircraft_Type__c'].str.contains('CJ')]
jj_opportunity_df['CreatedDate'] = pd.to_datetime(jj_opportunity_df['CreatedDate'])
#jj_opportunity_df = jj_opportunity_df[jj_opportunity_df['CreatedDate'].dt.year == 2025]
# year 2024 may provide rollover value for CJ2+
#jj_opportunity_df
hours_purchased_cj2 = jj_opportunity_df['Hours__c'].iloc[0]
hours_purchased_cj3 = jj_opportunity_df['Hours__c'].iloc[1]
print(hours_purchased_cj2)
print(hours_purchased_cj3)

25.0
25.0


In [13]:
# 11. Total Flight Time Used (from Fl3xx API, using provided flight IDs) - I don't think I am adding Legacy flight group
# Base on registration number, I should separate out CJ2+ and CJ3+ and add them up
# I should link to df_aircraft 
flight_time_FASP = df_ft['Flight Time (hours)'].iloc[1]
flight_time_FNAS = df_ft['Flight Time (hours)'].iloc[5]
print(flight_time_FASP)
print(flight_time_FNAS)
#CJ2+ : 1 hours used
#CJ3+ : 1.4 hours used
#I decided to not add the GASR to CJ2+ total flight time to keep consistency of my understanding

1.0
1.4


In [14]:
# Detailed Flight Breakdown for March 2025
df_invoices['INVDATE'] = pd.to_datetime(df_invoices['INVDATE'].astype(str), format='%Y%m%d')
df_march_2025 = df_invoices[df_invoices['INVDATE'].dt.month == 3]
df_march_2025 = df_march_2025[df_march_2025['INVDATE'].dt.year == 2025]
df_march_2025
# Filter df_march_2025 to include only rows with relevant flight IDs
df_march_2025_filtered = df_march_2025[df_march_2025['FLIGHTID'].isin(relevant_flight_ids)]
df_march_2025_filtered

,INVUNIQ,ORDNUMBER,CUSTOMER,BILNAME,COMMENT,INVDATE,INVNUMBER,APPROVELMT,FLIGHTID
994,1251973,ORD0058908,24882,JimJets,...,2025-03-12,IN00059551,5890.88,79952.0
1087,1253128,ORD0059000,24882,JimJets,Saturday use ...,2025-03-07,IN00059644,9350.60,79953.0
1099,1252746,ORD0058973,24882,JimJets,L-450 Flight ...,2025-03-16,IN00059618,3824.96,79956.0
1110,1253572,ORD0059027,24882,JimJets,...,2025-03-05,IN00059671,18610.59,79960.0
1234,1255238,ORD0059157,24882,JimJets,...,2025-03-13,IN00059799,3085.70,79959.0
1249,1255427,ORD0059171,24882,JimJets,...,2025-03-28,IN00059812,21497.20,79958.0


In [15]:
conn = sqlite3.connect('airsprint_data.db')
cursor = conn.cursor()
df_march_2025_filtered.to_sql('march_2025_filtered', conn, if_exists='replace', index=False)
conn.commit()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)
conn.close()

Tables in the database: [('flight_times',), ('aircraft',), ('march_2025_filtered',)]


In [16]:
# 1. Invoice# Check
# 2. Date of Invoice Check
# 3. Aircraft Registration # - merge df_march_2025_filtered with df_flight_times via FLIGHTID
# Reconnect to the database
conn = sqlite3.connect('airsprint_data.db')

# Define your SQL query
query = """
SELECT
    mf.*, 
    ft.* 
FROM 
    march_2025_filtered AS mf
INNER JOIN 
    flight_times AS ft
ON 
    mf.FLIGHTID = ft."Flight ID"
"""

# Run the query and load the result into a DataFrame
df_merged_a = pd.read_sql(query, conn)

# Display the result
print(df_merged_a)

# Close the connection
conn.close()

# 4. Total Flight time for each flight - Check
# 5. Origin and Destination Airports - Straight from API - Check
# 6. Total Invoice Amount - should be APPROVELMT - Check

   INVUNIQ               ORDNUMBER  CUSTOMER  BILNAME  \
0  1251973  ORD0058908                 24882  JimJets   
1  1253128  ORD0059000                 24882  JimJets   
2  1252746  ORD0058973                 24882  JimJets   
3  1253572  ORD0059027                 24882  JimJets   
4  1255238  ORD0059157                 24882  JimJets   
5  1255427  ORD0059171                 24882  JimJets   

                                             COMMENT              INVDATE  \
0                                                ...  2025-03-12 00:00:00   
1  Saturday use                                  ...  2025-03-07 00:00:00   
2  L-450 Flight                                  ...  2025-03-16 00:00:00   
3                                                ...  2025-03-05 00:00:00   
4                                                ...  2025-03-13 00:00:00   
5                                                ...  2025-03-28 00:00:00   

                INVNUMBER  APPROVELMT  FLIGHTID  Flight ID  

In [17]:
data = {
    'Account Name': [account_name],
    'Contact Name': [contact_name],
    'Account Number': [account_number],
    'Date': [today_date],
    'Anniversary Date': [anniversary_date_cj2],
    'Aircraft': [aircraft_cj2],
    'AirGroup': [airgroup_cj2],
    'Owner %' : [owner_cj2],
    'Hours Available At the Start of Current Year':[annual_hours_cj2],
    'Hours Purchased': [hours_purchased_cj2],
    'Flight Time Used in March': [flight_time_FASP],
    'Total Hours Remaining Until Anniversary Date': [annual_hours_cj2 + hours_purchased_cj2 - flight_time_FASP ]
}

datatwo = {
    'Account Name': [account_name],
    'Contact Name': [contact_name],
    'Account Number': [account_number],
    'Date': [today_date],
    'Anniversary Date': [anniversary_date_cj3],
    'Aircraft': [aircraft_cj3],
    'AirGroup': [airgroup_cj3],
    'Owner %' : [owner_cj3],
    'Hours Available At the Start of Current Year':[annual_hours_cj3],
    'Hours Purchased': [hours_purchased_cj3],
    'Flight Time Used in March': [flight_time_FNAS],
    'Total Hours Remaining Until Anniversary Date': [annual_hours_cj3 + hours_purchased_cj3 - flight_time_FNAS]
}
df = pd.DataFrame(data)
dftwo = pd.DataFrame(datatwo)
df['Account Number'] = df['Account Number'].astype(str)
dftwo['Account Number'] = df['Account Number'].astype(str)
df.to_csv('invoice_summary_cj2+.csv', index=False)
dftwo.to_csv('invoice_summary_cj3+.csv', index=False)

In [18]:
df_merged_a

,INVUNIQ,ORDNUMBER,CUSTOMER,BILNAME,COMMENT,INVDATE,INVNUMBER,APPROVELMT,FLIGHTID,Flight ID,ETD,ETA,Registration Number,Airport From,Airport To,Flight Time (hours)
0,1251973,ORD0058908,24882,JimJets,...,2025-03-12 00:00:00,IN00059551,5890.88,79952.0,79952,2025-03-12T15:00:00.000Z,2025-03-12T19:31:00.000Z,C-GASR,CYYC,CYYZ,4.5
1,1253128,ORD0059000,24882,JimJets,Saturday use ...,2025-03-07 00:00:00,IN00059644,9350.60,79953.0,79953,2025-03-07T19:00:00.000Z,2025-03-07T20:00:00.000Z,C-FASP,KSLC,KLAS,1.0
2,1252746,ORD0058973,24882,JimJets,L-450 Flight ...,2025-03-16 00:00:00,IN00059618,3824.96,79956.0,79956,2025-03-13T16:00:00.000Z,2025-03-13T16:24:00.000Z,C-FSRX,KLAS,CYVR,0.4
3,1253572,ORD0059027,24882,JimJets,...,2025-03-05 00:00:00,IN00059671,18610.59,79960.0,79960,2025-03-05T20:00:00.000Z,2025-03-05T21:29:00.000Z,C-FSRX,CYED,US-0222,1.5
4,1255238,ORD0059157,24882,JimJets,...,2025-03-13 00:00:00,IN00059799,3085.70,79959.0,79959,2025-03-13T07:00:00.000Z,2025-03-13T07:44:00.000Z,C-GBAS,None,KREG,0.7
5,1255427,ORD0059171,24882,JimJets,...,2025-03-28 00:00:00,IN00059812,21497.20,79958.0,79958,2025-03-28T23:00:00.000Z,2025-03-29T00:25:00.000Z,C-FNAS,CYXE,CYLW,1.4


In [19]:
#important columns
relevant_columns = ['INVNUMBER','INVDATE','Registration Number', 'Flight Time (hours)','Airport From','Airport To','APPROVELMT','COMMENT']
df_merged_b = df_merged_a[relevant_columns]
df_merged_b.rename(columns={
    'INVNUMBER': 'Invoice Number',
    'INVDATE': 'Invoice Date',
    'Airport From': 'Departure Airport',
    'Airport To': 'Arrival Airport',
    'APPROVELMT': 'Total Invoice Amount'
}, inplace=True)

/var/folders/x3/jv8gc_ln2_qgkg2mcy722mlh0000gn/T/ipykernel_16321/232141518.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_b.rename(columns={


In [20]:
df_merged_b.to_csv('invoice.csv', index=False)

In [21]:
import requests

flight_id = 'YOUR_flightId_PARAMETER'
url = 'https://test.fl3xx.com/api/external/flight/' + '79959'

headers = {'X-Auth-Token': '4y9lRGIoQtzDdPLphsO4rtzhTbLPk6Sg'}

response = requests.get(url, headers=headers)

data = response.json()
print(data)

{'flightId': 79959, 'quoteId': 109350, 'bookingReference': '109350', 'userReference': None, 'accountReference': None, 'flightNumber': '115S', 'flightNumberCompany': '14-115S', 'flightType': 'PAX', 'aircraftCategory': 'A270', 'registrationNumber': 'C-GBAS', 'blockOffEstUTC': '2025-03-13T07:00:00.000Z', 'blockOnEstUTC': '2025-03-13T07:44:00.000Z', 'blockOffEstLocal': '2025-03-12T21:00', 'blockOnEstLocal': '2025-03-13T02:44', 'airportFrom': None, 'airportTo': 'KREG', 'paxReferences': [], 'rescheduledDateFrom': None, 'rescheduledDateTo': None, 'realAirportFrom': None, 'realAirportTo': 'KREG', 'etd': '2025-03-13T07:00:00.000Z', 'eta': '2025-03-13T07:44:00.000Z', 'realDateOUT': None, 'realDateOFF': None, 'realDateON': None, 'realDateIN': None, 'customerId': '100067', 'accountTrigram': '2262', 'customerTrigram': '2262', 'customerFirstname': 'Tester', 'customerLastname': 'Aviation', 'accountId': 24882, 'accountName': 'JimJets', 'dateFrom': None, 'dateTo': None, 'flightStatus': 'None', 'fplType

In [22]:
# https://www.flightaware.com/live/flight/CGBAS/history/20250318/1315Z/KPBI/KRSW
# No idea what's going on with CGBAS with null Airport From & KREG (Louisiana) Airport To